In [76]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [2]:
df = pd.read_csv("tweets.csv")
stop_words = set(stopwords.words('english'))

In [3]:
df.head()

,user_id,user_key,created_at,created_str,retweet_count,retweeted,favorite_count,text,tweet_id,source,hashtags,expanded_urls,posted,mentions,retweeted_status_id,in_reply_to_status_id
0,2.532612e+09,kathiemrr,1.488207e+12,2017-02-27 14:54:00,NaN,NaN,NaN,#ThingsDoneByMistake kissing auntie in the lips,8.362279e+17,NaN,"[""ThingsDoneByMistake""]",[],POSTED,[],NaN,NaN
1,2.531160e+09,traceyhappymom,1.471273e+12,2016-08-15 14:50:20,NaN,NaN,NaN,RT @mc_derpin: #TheOlderWeGet the more pessimi...,7.651989e+17,NaN,"[""TheOlderWeGet""]",[],POSTED,[],NaN,NaN
2,NaN,evewebster373,1.435701e+12,2015-06-30 21:56:09,NaN,NaN,NaN,RT @dmataconis: Ready To Feel Like A Failure? ...,6.160023e+17,NaN,[],[],POSTED,[],NaN,NaN
3,4.840552e+09,blacktolive,1.474013e+12,2016-09-16 08:04:48,18.0,False,17.0,Amen! #blacklivesmatter https://t.co/wGffaOqgzl,7.766933e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""Blacklivesmatter""]",[],POSTED,[],NaN,NaN
4,1.694026e+09,jacquelinisbest,1.474228e+12,2016-09-18 19:46:25,0.0,False,0.0,RT @NahBabyNah: Twitchy: Chuck Todd caught out...,7.775946e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""WakeUpAmerica""]","[""http://ln.is/twitchy.com/loriz-31/3yafU""]",POSTED,"[""nahbabynah""]",7.775915e+17,NaN


In [4]:
# Show how to get sentiment using TextBlob
from textblob import TextBlob
for i,row in df.head().iterrows():
    print("Text:",row['text'],"\nSentiment:",TextBlob(row['text']).sentiment.polarity)

Text: #ThingsDoneByMistake kissing auntie in the lips 
Sentiment: 0.0
Text: RT @mc_derpin: #TheOlderWeGet the more pessimistic we are https://t.co/zS3jHZJl8P 
Sentiment: 0.5
Text: RT @dmataconis: Ready To Feel Like A Failure? Joan Of Arc Was Only 19 When She Was Burned At The Stake http://t.co/S2j1IFm4y9 
Sentiment: -0.038888888888888896
Text: Amen! #blacklivesmatter https://t.co/wGffaOqgzl 
Sentiment: 0.0
Text: RT @NahBabyNah: Twitchy: Chuck Todd caught out there shilling for Hillary Clinton
The post BUSTED: Adam Baldwi... https://t.co/ay28pMpDw6 #… 
Sentiment: 0.0


In [5]:
df.shape

(203451, 16)

In [6]:
# Create an array of all sentiments for the dataframe
sentiment = [TextBlob(str(row['text'])).sentiment.polarity for i,row in df.iterrows()]

In [7]:
# Add the sentiment column to the dataframe with the sentiment array as the values
df['sentiment'] = sentiment

In [8]:
# Scroll all the way to the right to see the sentiment column added
df.head()

,user_id,user_key,created_at,created_str,retweet_count,retweeted,favorite_count,text,tweet_id,source,hashtags,expanded_urls,posted,mentions,retweeted_status_id,in_reply_to_status_id,sentiment
0,2.532612e+09,kathiemrr,1.488207e+12,2017-02-27 14:54:00,NaN,NaN,NaN,#ThingsDoneByMistake kissing auntie in the lips,8.362279e+17,NaN,"[""ThingsDoneByMistake""]",[],POSTED,[],NaN,NaN,0.000000
1,2.531160e+09,traceyhappymom,1.471273e+12,2016-08-15 14:50:20,NaN,NaN,NaN,RT @mc_derpin: #TheOlderWeGet the more pessimi...,7.651989e+17,NaN,"[""TheOlderWeGet""]",[],POSTED,[],NaN,NaN,0.500000
2,NaN,evewebster373,1.435701e+12,2015-06-30 21:56:09,NaN,NaN,NaN,RT @dmataconis: Ready To Feel Like A Failure? ...,6.160023e+17,NaN,[],[],POSTED,[],NaN,NaN,-0.038889
3,4.840552e+09,blacktolive,1.474013e+12,2016-09-16 08:04:48,18.0,False,17.0,Amen! #blacklivesmatter https://t.co/wGffaOqgzl,7.766933e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""Blacklivesmatter""]",[],POSTED,[],NaN,NaN,0.000000
4,1.694026e+09,jacquelinisbest,1.474228e+12,2016-09-18 19:46:25,0.0,False,0.0,RT @NahBabyNah: Twitchy: Chuck Todd caught out...,7.775946e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""WakeUpAmerica""]","[""http://ln.is/twitchy.com/loriz-31/3yafU""]",POSTED,"[""nahbabynah""]",7.775915e+17,NaN,0.000000


In [9]:
for i in range(20):
    print("Text:",df.iloc[i,7],"Sentiment:",TextBlob(df.iloc[i,7]).sentiment.polarity,"Hashtag Sentiment:",TextBlob(str(df.iloc[i,10])).sentiment.polarity)

Text: #ThingsDoneByMistake kissing auntie in the lips Sentiment: 0.0 Hashtag Sentiment: 0.0
Text: RT @mc_derpin: #TheOlderWeGet the more pessimistic we are https://t.co/zS3jHZJl8P Sentiment: 0.5 Hashtag Sentiment: 0.0
Text: RT @dmataconis: Ready To Feel Like A Failure? Joan Of Arc Was Only 19 When She Was Burned At The Stake http://t.co/S2j1IFm4y9 Sentiment: -0.038888888888888896 Hashtag Sentiment: 0.0
Text: Amen! #blacklivesmatter https://t.co/wGffaOqgzl Sentiment: 0.0 Hashtag Sentiment: 0.0
Text: RT @NahBabyNah: Twitchy: Chuck Todd caught out there shilling for Hillary Clinton
The post BUSTED: Adam Baldwi... https://t.co/ay28pMpDw6 #… Sentiment: 0.0 Hashtag Sentiment: 0.0
Text: RT @mcicero10: #BernieSanders #Trump people should rally TOGETHER against the establishment who is 💩-ing on both choices #thefix Sentiment: 0.0 Hashtag Sentiment: 0.0
Text: RT @ItsJustJaynie: @HillaryClinton The undecided voters on that stage was polled and said trump won. @cnn is biased. Sentiment: 0.0 Hashta

In [10]:
regex = re.compile(r'([A-Z][a-z]*)')

In [11]:
separated_hashtags = [' '.join(re.split(regex, re.sub(r'[\W]','',df.iloc[i,10]))).strip(" ") for i in range(df.shape[0])]

In [14]:
for i in range(5):
    print(separated_hashtags[i])

Things  Done  By  Mistake
The  Older  We  Get

Blacklivesmatter
Wake  Up  America


In [12]:
for i in range(20):
    print("Sentiment:",TextBlob(separated_hashtags[i]).sentiment.polarity)

Sentiment: 0.0
Sentiment: 0.16666666666666666
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.13636363636363635
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0
Sentiment: 0.0


In [134]:
stop_words.add('rt')
stop_words.add('@')

In [135]:
stop_words

{'@',
 'RT',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over'

In [149]:
text = df['text']
cleaned_text = []
wordnet_lemmatizer = WordNetLemmatizer()
link_regex = re.compile(r'((?:http(?:s)?):\/\/(?:w{3}|t)\.(?:\w+\.)?(?:co(?:m)?)?(?:\/)(?:\w+)?)')
punctuation = re.compile(r'\.+|,+|\?+|#|:|!+')
for row in text.head():
    row = link_regex.sub('',row.lower())
    tokens = word_tokenize(row.lower())
    words_to_add = []
    for word in tokens:
        word = punctuation.sub('',word)
        if word not in stop_words:
            words_to_add.append(wordnet_lemmatizer.lemmatize(word))
    cleaned_text.append(words_to_add)

In [148]:
cleaned_text

[['#', 'thingsdonebymistake', 'kissing', 'auntie', 'lip'],
 ['mc_derpin', ':', '#', 'theolderweget', 'pessimistic'],
 ['dmataconis',
  ':',
  'ready',
  'feel',
  'like',
  'failure',
  '?',
  'joan',
  'arc',
  '19',
  'burned',
  'stake'],
 ['amen', '!', '#', 'blacklivesmatter'],
 ['nahbabynah',
  ':',
  'twitchy',
  ':',
  'chuck',
  'todd',
  'caught',
  'shilling',
  'hillary',
  'clinton',
  'post',
  'busted',
  ':',
  'adam',
  'baldwi',
  '...',
  '#',
  '…']]